In [36]:
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
%run ./tools/test_net.py --gpu 0 --def models/pascal_voc/ResNet-50/rfcn_end2end/test_agnostic.prototxt --net /home/ubuntu/py-R-FCN/output/rfcn_end2end_ohem/voc_2007_trainval/resnet50_rfcn_ohem_iter_20000.caffemodel --imdb voc_2007_test --cfg experiments/cfgs/rfcn_end2end_ohem.yml

Called with args:
Namespace(caffemodel='/home/ubuntu/py-R-FCN/output/rfcn_end2end_ohem/voc_2007_trainval/resnet50_rfcn_ohem_iter_20000.caffemodel', cfg_file='experiments/cfgs/rfcn_end2end_ohem.yml', comp_mode=False, gpu_id=0, imdb_name='voc_2007_test', max_per_image=400, prototxt='models/pascal_voc/ResNet-50/rfcn_end2end/test_agnostic.prototxt', rpn_file=None, set_cfgs=None, vis=False, wait=True)


In [2]:
import cPickle  
with open('output/rfcn_end2end_ohem/voc_2007_test/resnet50_rfcn_ohem_iter_20000/test_preds.pkl','rb') as f:
    test_preds = cPickle.load(f)  

In [11]:
submission = pd.DataFrame(test_preds, columns=FISH_CLASSES)
#submission.loc[:, 'image'] = pd.Series(test_files, index=submission.index)
submission.insert(0, 'image', test_files)

now = datetime.datetime.now()
info = 'Baseline_' + '{:.4f}'.format(min(val_losses))
sub_file = 'submission_' + info + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
submission.to_csv(sub_file, index=False)

8

In [34]:
THRESH = 0.5
info_threshed = []
for im in range(1000):
    info_threshed_im = []
    for c in range(1,8):
        for s in range(len(info[c][im])):
            if info[c][im][s][-1] >= 0.5:
                info_threshed_im.append([c, info[c][im][s][-1]])
    info_threshed.append([im, info_threshed_im])          
    